<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/jangjaewon/20221015_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys
from datetime import datetime, timedelta

In [ ]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


['CHANGELOG.txt',
 'LICENSE.txt',
 'SHA256SUMS.txt',
 'index.html',
 'icu',
 'hosp',
 'drop_2_chartevents.csv']

In [ ]:
admission = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv', low_memory=False, dtype='str', keep_default_na=False)
patients = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/patients.csv', low_memory=False, dtype='str', keep_default_na=False)
diagnoses_icd = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/diagnoses_icd.csv', low_memory=False, dtype='str', keep_default_na=False)

In [ ]:
microbiologyevents = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/microbiologyevents.csv', low_memory=False, dtype='str', keep_default_na=False,chunksize = 10 ** 5)
chartevents = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/icu/chartevents.csv', low_memory=False, dtype='str', keep_default_na=False,chunksize = 10 ** 5)
labevents = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/labevents.csv', low_memory=False, dtype='str', keep_default_na=False,chunksize = 10 ** 5)

In [ ]:
print(admission.size)
print(patients.size)
print(diagnoses_icd.size)

6814860
1892760
25034420


In [ ]:
next(microbiologyevents).columns

Index(['microevent_id', 'subject_id', 'hadm_id', 'micro_specimen_id',
       'chartdate', 'charttime', 'spec_itemid', 'spec_type_desc', 'test_seq',
       'storedate', 'storetime', 'test_itemid', 'test_name', 'org_itemid',
       'org_name', 'isolate_num', 'quantity', 'ab_itemid', 'ab_name',
       'dilution_text', 'dilution_comparison', 'dilution_value',
       'interpretation', 'comments'],
      dtype='object')

In [ ]:
next(microbiologyevents).head(5)

,microevent_id,subject_id,hadm_id,micro_specimen_id,chartdate,charttime,spec_itemid,spec_type_desc,test_seq,storedate,...,org_name,isolate_num,quantity,ab_itemid,ab_name,dilution_text,dilution_comparison,dilution_value,interpretation,comments
700000,700001,12059869,,3024505,2120-02-05 00:00:00,2120-02-05 10:55:00,70017,SEROLOGY/BLOOD,1,2120-02-06 00:00:00,...,,,,,,,,,,POSITIVE BY EIA. A positive IgG result genera...
700001,700002,12059869,,3944665,2120-02-05 00:00:00,2120-02-05 10:55:00,70017,SEROLOGY/BLOOD,1,2120-02-07 00:00:00,...,,,,,,,,,,POSITIVE BY EIA. A positive IgG result genera...
700002,700003,12059869,,7687251,2120-02-05 00:00:00,2120-02-05 10:55:00,70017,SEROLOGY/BLOOD,1,2120-02-06 00:00:00,...,,,,,,,,,,NONREACTIVE. Reference Range: Non-Reactive.
700003,700004,12059869,,8423827,2120-02-05 00:00:00,2120-02-05 10:55:00,70017,SEROLOGY/BLOOD,1,2120-02-06 00:00:00,...,,,,,,,,,,POSITIVE BY EIA. A positive IgG result genera...
700004,700005,12059869,,4619203,2120-02-06 00:00:00,,70064,STOOL,1,2120-02-08 00:00:00,...,,,,,,,,,,NO SALMONELLA OR SHIGELLA FOUND.


In [ ]:
microbiologyevents_1 = next(microbiologyevents)

In [ ]:
microbiologyevents_1 = pd.merge(left = microbiologyevents_1, right = sepsis[['subject_id','culture_time','sepsis3']], how='inner', left_on=['subject_id','charttime'], right_on=['subject_id','culture_time'])

In [ ]:
microbiologyevents_1 = microbiologyevents_1[microbiologyevents_1['spec_type_desc'] == 'BLOOD CULTURE']

In [ ]:
microbiologyevents_1['org_name'].value_counts() # 박테리아가 발견되면 박테리아의 각 유기체가 에 존재 org_name하여 단일 표본에 대해 여러 행이 나타납니다

                                                         593
ESCHERICHIA COLI                                         298
KLEBSIELLA PNEUMONIAE                                     59
STAPHYLOCOCCUS, COAGULASE NEGATIVE                        45
PROTEUS MIRABILIS                                         34
PSEUDOMONAS AERUGINOSA                                    28
STAPH AUREUS COAG +                                       24
ENTEROCOCCUS FAECALIS                                     23
STAPHYLOCOCCUS EPIDERMIDIS                                22
STREPTOCOCCUS ANGINOSUS (MILLERI) GROUP                   16
KLEBSIELLA OXYTOCA                                        12
MORGANELLA MORGANII                                        9
ENTEROBACTER CLOACAE COMPLEX                               9
ENTEROCOCCUS CASSELIFLAVUS                                 5
ENTEROCOCCUS FAECIUM                                       5
VIRIDANS STREPTOCOCCI                                      3
CORYNEBACTERIUM SPECIES 

In [ ]:
microbiologyevents_1['ab_name'].value_counts() # 항생제가 주어진 박테리아 유기체에 대해 테스트된다면, 테스트된 각 항생제는 열에 존재할 것입니다 (즉, 주어진 것과 연관된 ab_name주어진 것에 대한 여러 행 ). 

                        640
GENTAMICIN               48
TRIMETHOPRIM/SULFA       40
CEFTAZIDIME              40
TOBRAMYCIN               39
CIPROFLOXACIN            39
PIPERACILLIN/TAZO        39
CEFEPIME                 39
MEROPENEM                39
CEFTRIAXONE              35
AMPICILLIN               35
AMPICILLIN/SULBACTAM     33
CEFAZOLIN                31
VANCOMYCIN               20
CLINDAMYCIN              13
LEVOFLOXACIN             13
ERYTHROMYCIN             12
PENICILLIN G             11
OXACILLIN                 9
DAPTOMYCIN                8
RIFAMPIN                  8
TETRACYCLINE              5
AMIKACIN                  5
CEFUROXIME                4
PIPERACILLIN              2
LINEZOLID                 1
Name: ab_name, dtype: int64

In [ ]:
admission.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag'],
      dtype='object')

In [ ]:
sepsis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/sepsis3.csv', low_memory=False, dtype='str', keep_default_na=False)

In [ ]:
sepsis.columns

Index(['subject_id', 'stay_id', 'antibiotic_time', 'culture_time',
       'suspected_infection_time', 'sofa_time', 'sofa_score', 'respiration',
       'coagulation', 'liver', 'cardiovascular', 'cns', 'renal', 'sepsis3'],
      dtype='object')

In [ ]:
sepsis

,subject_id,stay_id,antibiotic_time,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,16266622,32828424,2134-09-20 16:00:00,2134-09-20 15:32:00,2134-09-20 15:32:00,2134-09-20 16:00:00,2,2,0,0,0,0,0,true
1,13830137,31445489,2174-05-06 22:00:00,2174-05-06 00:00:00,2174-05-06 00:00:00,2174-05-06 21:00:00,2,0,0,0,0,0,2,true
2,14367208,39300189,2135-09-14 18:00:00,2135-09-14 00:45:00,2135-09-14 00:45:00,2135-09-14 04:00:00,2,0,0,1,0,0,1,true
3,12035173,34909617,2168-01-25 02:00:00,2168-01-22 04:04:00,2168-01-22 04:04:00,2168-01-22 01:00:00,2,0,0,0,0,2,0,true
4,13729279,39470047,2181-01-14 04:00:00,2181-01-13 17:50:00,2181-01-13 17:50:00,2181-01-14 05:00:00,2,0,0,0,0,2,0,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34673,10827578,34763877,2130-05-09 08:00:00,2130-05-09 11:55:00,2130-05-09 08:00:00,2130-05-07 08:00:00,17,3,3,3,4,0,4,true
34674,11713814,37767813,2159-05-31 08:00:00,2159-05-31 06:25:00,2159-05-31 06:25:00,2159-05-29 07:00:00,17,2,2,2,4,4,3,true
34675,17715294,37357219,2131-07-25 13:00:00,2131-07-25 12:50:00,2131-07-25 12:50:00,2131-07-23 13:00:00,17,2,4,4,4,0,3,true
34676,11180969,35240504,2139-10-30 08:00:00,2139-10-29 21:52:00,2139-10-29 21:52:00,2139-10-27 22:00:00,20,4,2,2,4,4,4,true


In [ ]:
admission = pd.merge(left = admission, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
patients = pd.merge(left = patients, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
diagnoses_icd = pd.merge(left = diagnoses_icd, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
#microbiologyevents = pd.merge(left = microbiologyevents, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
#chartevents = pd.merge(left = chartevents, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
#labevents = pd.merge(left = labevents, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')

In [ ]:
microbiologyevents = pd.merge(left = pd.concat(microbiologyevents), right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')

In [ ]:
#chartevents = pd.merge(left = pd.concat(chartevents), right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
#labevents = pd.merge(left = pd.concat(labevents), right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')

In [ ]:
microbiologyevents

,microevent_id,subject_id,hadm_id,micro_specimen_id,chartdate,charttime,spec_itemid,spec_type_desc,test_seq,storedate,...,isolate_num,quantity,ab_itemid,ab_name,dilution_text,dilution_comparison,dilution_value,interpretation,comments,sepsis3
0,631,10001884,,802664,2122-02-12 00:00:00,2122-02-12 12:30:00,70012,BLOOD CULTURE,1,2122-02-18 00:00:00,...,,,,,,,,,NO GROWTH.,true
1,632,10001884,,802664,2122-02-12 00:00:00,2122-02-12 12:30:00,70012,BLOOD CULTURE,2,2122-02-18 00:00:00,...,,,,,,,,,NO GROWTH.,true
2,633,10001884,,8339529,2125-11-30 00:00:00,2125-11-30 15:26:00,70012,BLOOD CULTURE,1,2125-12-06 00:00:00,...,,,,,,,,,NO GROWTH.,true
3,634,10001884,,3187447,2125-11-30 00:00:00,2125-11-30 18:40:00,70012,BLOOD CULTURE,1,2125-12-06 00:00:00,...,,,,,,,,,NO GROWTH.,true
4,635,10001884,,2893463,2125-12-01 00:00:00,,70042,Influenza A/B by DFA,1,2125-12-01 00:00:00,...,1,,,,,,,,,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488131,3395205,19999840,21033226,5323018,2164-09-14 00:00:00,2164-09-14 12:13:00,70091,MRSA SCREEN,1,2164-09-16 00:00:00,...,,,,,,,,,No MRSA isolated.,true
2488132,3395206,19999840,21033226,9959252,2164-09-17 00:00:00,2164-09-17 06:10:00,70064,STOOL,1,2164-09-17 00:00:00,...,,,,,,,,,Feces negative for C. difficile toxin A & B by...,true
2488133,3395207,19999840,21033226,186839,2164-09-17 00:00:00,2164-09-17 11:58:00,70012,BLOOD CULTURE,1,2164-09-23 00:00:00,...,,,,,,,,,NO GROWTH.,true
2488134,3395208,19999840,,43002,2164-09-18 00:00:00,,70055,POSTMORTEM CULTURE,1,2164-09-22 00:00:00,...,1,,,,,,,,,true


In [ ]:
# item id, valuenum 전처리해야함.

In [ ]:
sepsis.columns

Index(['subject_id', 'stay_id', 'antibiotic_time', 'culture_time',
       'suspected_infection_time', 'sofa_time', 'sofa_score', 'respiration',
       'coagulation', 'liver', 'cardiovascular', 'cns', 'renal', 'sepsis3'],
      dtype='object')

In [ ]:
sepsis

,subject_id,stay_id,antibiotic_time,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,16266622,32828424,2134-09-20 16:00:00,2134-09-20 15:32:00,2134-09-20 15:32:00,2134-09-20 16:00:00,2,2,0,0,0,0,0,true
1,13830137,31445489,2174-05-06 22:00:00,2174-05-06 00:00:00,2174-05-06 00:00:00,2174-05-06 21:00:00,2,0,0,0,0,0,2,true
2,14367208,39300189,2135-09-14 18:00:00,2135-09-14 00:45:00,2135-09-14 00:45:00,2135-09-14 04:00:00,2,0,0,1,0,0,1,true
3,12035173,34909617,2168-01-25 02:00:00,2168-01-22 04:04:00,2168-01-22 04:04:00,2168-01-22 01:00:00,2,0,0,0,0,2,0,true
4,13729279,39470047,2181-01-14 04:00:00,2181-01-13 17:50:00,2181-01-13 17:50:00,2181-01-14 05:00:00,2,0,0,0,0,2,0,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34673,10827578,34763877,2130-05-09 08:00:00,2130-05-09 11:55:00,2130-05-09 08:00:00,2130-05-07 08:00:00,17,3,3,3,4,0,4,true
34674,11713814,37767813,2159-05-31 08:00:00,2159-05-31 06:25:00,2159-05-31 06:25:00,2159-05-29 07:00:00,17,2,2,2,4,4,3,true
34675,17715294,37357219,2131-07-25 13:00:00,2131-07-25 12:50:00,2131-07-25 12:50:00,2131-07-23 13:00:00,17,2,4,4,4,0,3,true
34676,11180969,35240504,2139-10-30 08:00:00,2139-10-29 21:52:00,2139-10-29 21:52:00,2139-10-27 22:00:00,20,4,2,2,4,4,4,true


In [ ]:
sum(sepsis['sepsis3']=='true')/len(sepsis['sepsis3'])

1.0

In [ ]:
# 한 환자가 있고 입원 여러번 한 경우 입원 아이디 여러번
# stayid 가 다르다.
# syayid 첫번째꺼
# intime, outtime, death time
# intime과 outtime 안에 death time 이 있으면 상태악화로 정의

In [ ]:
# 제외조건


In [ ]:
# input data 붙여오기

In [ ]:
VITAL_SIGNS = {'PULSE': ['220045', '211'],
               'RESP': ['220210', '618'],
               'SBP': ['220050', '220179', '225309', '51', '455'],
               'DBP': ['220051', '220180', '225310', '8368', '8441'],
               'FiO2': ['223835', '189', '190'],
               'MAP': ['220052', '1321', '438']
               }
LABS = {'BILIRUBIN': ['50885'],
        'LACTATE': ['50813'],
        'pH': ['50820', '50831'],
        'SODIUM': ['50983'],
        'POTASSIUM': ['50971'],
        'CREATININE': ['50912'],
        'WBC': ['51301'],
        'HEMATOCRIT': ['51221']
        }

VITAL_SIGNS = {'PULSE': ['220045', '211'],
               'RESP': ['220210', '618'],
               'SBP': ['228152','229899','220050', '220179', '225309', '51', '455','224167','227243'],
               'DBP': ['220051', '220180', '225310', '8368', '8441'],
               'FiO2': ['229280','229841','226754','227009','227010', '223835', '189', '190'],
               'MAP': ['220052', '1321', '438']
               }

LABS = {'lactate': ['225668'],
        'glucose'	: ['220621','225664','226537','228388'],
        'creatinine' : ['220615','229761'],
        'WBC'	: ['220546','51301'],
        'heart rate' : ['220046','220047','226764','220045','227018'],
        'sodium' : ['220645','226534','228389','228390'],
        'temperature' : ['223762','226329','227632','227634'],
        'platelet' : ['227457'],
        'hemoglobin' : ['220228'],
        'pao2' : ['220224','227516','227039'],
}